***
*Project:* Free Energy Synchronization

*Author:* Jingwei Liu (Music department, UC San Diego)
***

# <span style="background-color:darkorange; color:white; padding:2px 6px">Tutorial</span> 


# 12 Tone First Species Counterpoint

### Generating Rules:

Rules for melodic lines:
1. No note may be followed by the same note
2. Each note may appear a maximum of 3 times in length of 10
3. No more than 2 leaps larger than a 4th (>= 5) in length of 10
4. Leaps larger than a 3rd (>=5) should be followed by a change of direction and a step-wise motion (1 or 2)

Numerical restriction:
5. Cantus range: G2 - C5 (43 - 72); Counterpoint range:  G3 - C6 (55 - 84)
6. Valid melodic intervals: m2(1), M2(2), m3(3), M3(4), P4(5), P5(7), m6(8), P8(12)
7. Valid harmonic intervals: m3(3), M3(4), P5(7), m6(8), M6(9), P8(12)

Rules for harmonic counters:
8. Max of 3 of the same harmonic interval in length of 10
9. A perfect interval (P5 or P8) must be approached with contrary motion and at least one of the voices moving by step

In [1]:
import numpy as np
import pandas as pd
import csv
from scamp import *
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense,Flatten,Softmax,Input

In [10]:
n = 10;
cantus_range_list = np.arange(43,73)
counter_range_list = np.arange(55,85)
melodic_interval = np.array([1,2,3,4,5,7,8,12])
melodic_interval_small = np.array([1,2,3,4])
harmonic_interval = np.array([3,4,7,8,9,12])
cantus_lst = [];
counter_lst = [];
st = np.random.choice(cantus_range_list)
cantus_lst.append(st)
counter_lst.append(np.random.choice(st + harmonic_interval))

for i in range(n):
    num = np.shape(cantus_lst)[0]
    cantus_choose_lst = np.concatenate((cantus_lst[-1] + melodic_interval, cantus_lst[-1] - melodic_interval))
    cantus_choose_lst = np.maximum(cantus_choose_lst,43)
    cantus_choose_lst = np.minimum(cantus_choose_lst,72)
    cantus_choose_lst = np.unique(cantus_choose_lst)

    counter_choose_lst = np.concatenate((counter_lst[-1] + melodic_interval, counter_lst[-1] - melodic_interval))
    counter_choose_lst = np.maximum(counter_choose_lst,55)
    counter_choose_lst = np.minimum(counter_choose_lst,84)
    counter_choose_lst = np.unique(counter_choose_lst)

    harmonic_diff = np.array(counter_lst) - np.array(cantus_lst)
    harmonic_diff = harmonic_diff[-10:]

    lst = []
    for j in cantus_choose_lst:
        for k in counter_choose_lst:
            if k-j in harmonic_interval and np.shape(np.where(harmonic_diff == k-j))[1] < 3:
                if k-j != 7 and k-j != 12:
                    lst.append([j,k])
                else:
                    if (k - counter_lst[-1])*(j - cantus_lst[-1]) < 0 and (np.abs(k - counter_lst[-1]) <= 2 or np.abs(j - cantus_lst[-1]) <= 2):
                        lst.append([j,k])
    if lst != []:
        save_lst = lst
    else:
        save_lst = [[np.random.choice(cantus_choose_lst),np.random.choice(counter_choose_lst)]]
    
    if num == 1:
        indes = np.random.randint(len(lst), size = 1)[0]
        [a,b] = lst[indes]
        cantus_lst.append(a)
        counter_lst.append(b)     
    else:
        cantus_dif_last = cantus_lst[-2] - cantus_lst[-1]
        counter_dif_last = counter_lst[-2] - counter_lst[-1]
        if np.abs(cantus_dif_last) >= 5:
            sign = np.sign(cantus_dif_last)
            a = cantus_lst[-1] + sign*1
            b = cantus_lst[-1] + sign*2
            cantus = np.array(lst)[:,0]
            Ind = [];
            for index in range(len(cantus)):
                if cantus[index] != a and cantus[index] != b:
                    Ind.append(index)
            lst = np.delete(lst, Ind, axis=0)
        if np.abs(counter_dif_last) >= 5:
            sign = np.sign(counter_dif_last)
            a = counter_lst[-1] + sign*1
            b = counter_lst[-1] + sign*2
            counter = np.array(lst)[:,1]
            Ind = [];
            for index in range(len(counter)):
                if counter[index] != a and counter[index] != b:
                    Ind.append(index)
            lst = np.delete(lst, Ind, axis=0)
            
        cantus_diff = np.diff(cantus_lst)[-10:]
        counter_diff = np.diff(counter_lst)[-10:]
        
        cantus_num_leap = np.shape(np.where(np.abs(cantus_diff) >= 5))[1]
        counter_num_leap = np.shape(np.where(np.abs(counter_diff) >= 5))[1]
        
        if cantus_num_leap >= 2:
            cantus = np.array(lst)[:,0]
            Ind = [];
            for index in range(len(cantus)):
                if np.abs(cantus[index] - cantus_lst[-1]) >= 5:
                    Ind.append(index)
            lst = np.delete(lst, Ind, axis=0)
        if counter_num_leap >= 2:
            counter = np.array(lst)[:,1]
            Ind = [];
            for index in range(len(counter)):
                if np.abs(counter[index] - counter_lst[-1]) >= 5:
                    Ind.append(index)
            lst = np.delete(lst, Ind, axis=0)
        
        cantus = np.array(lst)[:,0]
        Ind = [];
        for index in range(len(cantus)):
            if np.shape(np.where(cantus_lst[-10:] == cantus[index]))[1] >= 3:
                Ind.append(index)
        lst = np.delete(lst, Ind, axis=0)
        
        counter = np.array(lst)[:,1]
        Ind = [];
        for index in range(len(counter)):
            if np.shape(np.where(counter_lst[-10:] == counter[index]))[1] >= 3:
                Ind.append(index)
        lst = np.delete(lst, Ind, axis=0)
        
        if len(lst) == 0:
            lst = save_lst

        indes = np.random.randint(len(lst), size = 1)[0]
        [a,b] = lst[indes]
        cantus_lst.append(a)
        counter_lst.append(b)

In [11]:
cantus_lst

[58,
 66,
 64,
 68,
 66,
 58,
 59,
 57,
 58,
 57,
 56,
 55,
 62,
 60,
 59,
 62,
 65,
 64,
 65,
 62,
 54,
 55,
 52,
 54,
 56,
 58,
 46,
 48,
 49,
 48,
 52,
 51,
 56,
 54,
 51,
 53,
 55,
 56,
 63,
 62,
 60,
 62,
 66,
 63,
 55,
 56,
 53,
 50,
 51,
 46,
 48,
 51,
 52,
 53,
 55,
 54,
 53,
 60,
 59,
 60,
 61,
 57,
 61,
 54,
 55,
 57,
 54,
 56,
 59,
 57,
 49,
 50,
 53,
 50,
 53,
 52,
 51,
 49,
 54,
 52,
 51,
 54,
 58,
 56,
 48,
 49,
 51,
 54,
 53,
 49,
 50,
 52,
 47,
 48,
 46,
 48,
 46,
 58,
 56,
 60,
 59,
 62,
 61,
 64,
 61,
 53,
 54,
 55,
 62,
 60,
 59,
 61,
 59,
 61,
 60,
 57,
 55,
 63,
 62,
 64,
 69,
 68,
 64,
 60,
 62,
 60,
 63,
 60,
 67,
 66,
 62,
 67,
 66,
 67,
 70,
 71,
 70,
 68,
 66,
 59,
 61,
 58,
 50,
 51,
 54,
 53,
 55,
 54,
 56,
 54,
 59,
 58,
 60,
 61,
 59,
 67,
 69,
 65,
 63,
 62,
 65,
 58,
 60,
 58,
 54,
 52,
 56,
 52,
 50,
 51,
 56,
 55,
 51,
 58,
 56,
 59,
 55,
 57,
 54,
 56,
 58,
 61,
 59,
 60,
 62,
 63,
 62,
 63,
 68,
 66,
 59,
 61,
 57,
 56,
 58,
 55,
 51,
 54,
 52,
 53,


In [12]:
counter_lst

[67,
 69,
 71,
 76,
 74,
 66,
 68,
 66,
 62,
 64,
 60,
 64,
 65,
 64,
 71,
 70,
 73,
 72,
 68,
 65,
 58,
 59,
 61,
 63,
 60,
 61,
 55,
 56,
 57,
 60,
 59,
 63,
 64,
 63,
 55,
 57,
 58,
 60,
 72,
 71,
 68,
 70,
 69,
 66,
 67,
 63,
 65,
 58,
 60,
 55,
 57,
 59,
 55,
 57,
 59,
 62,
 65,
 63,
 71,
 69,
 64,
 66,
 65,
 66,
 64,
 60,
 61,
 60,
 63,
 64,
 57,
 58,
 61,
 62,
 57,
 59,
 60,
 58,
 57,
 55,
 58,
 57,
 62,
 60,
 56,
 58,
 59,
 62,
 57,
 58,
 57,
 55,
 56,
 55,
 58,
 56,
 58,
 66,
 64,
 69,
 68,
 66,
 65,
 68,
 64,
 65,
 61,
 63,
 65,
 67,
 68,
 70,
 63,
 64,
 68,
 69,
 64,
 66,
 69,
 72,
 73,
 71,
 72,
 64,
 66,
 67,
 66,
 68,
 76,
 75,
 71,
 75,
 78,
 74,
 73,
 75,
 74,
 72,
 69,
 71,
 68,
 61,
 62,
 60,
 62,
 65,
 63,
 62,
 59,
 61,
 68,
 67,
 63,
 64,
 68,
 75,
 73,
 69,
 71,
 70,
 68,
 70,
 69,
 62,
 63,
 64,
 59,
 60,
 59,
 58,
 60,
 63,
 59,
 61,
 68,
 66,
 64,
 60,
 61,
 64,
 67,
 64,
 71,
 69,
 66,
 71,
 70,
 72,
 71,
 70,
 67,
 64,
 66,
 68,
 61,
 62,
 63,
 58,
 60,
 61,


In [13]:
s = Session()
s.tempo = 180
piano1 = s.new_part("piano")
piano2 = s.new_part("piano")

def cantus():
    for i in cantus_lst[1000:1020]:
        piano2.play_note(i,1,4)
        
def counter():
    for i in counter_lst[1000:1020]:
        piano1.play_note(i,1,4)
        
        
s.fast_forward_to_beat(100)

s.start_transcribing()
s.fork(counter)
s.fork(cantus)
s.wait_for_children_to_finish()
performance = s.stop_transcribing()
performance.to_score(title = "First Species Counterpoint", composer = "My programme",time_signature = "4/4").show_xml()

Using preset Piano Merlin for piano
Using preset Piano Merlin for piano


In [14]:
np.array(counter_lst[1000:1020]) - np.array(cantus_lst[1000:1020])

array([ 3,  4,  8,  9,  9,  8,  9,  3,  7, 12,  3,  7,  4, 12,  8,  3,  7,
       12,  3,  9])

In [15]:
np.diff(counter_lst[1000:1020])

array([ 4,  4, -3,  3, -3,  2, -5,  2,  1, -8,  1, -2,  1, -2, -3,  1,  1,
       -4,  5])

In [16]:
np.diff(counter_lst[1000:1020])

array([ 4,  4, -3,  3, -3,  2, -5,  2,  1, -8,  1, -2,  1, -2, -3,  1,  1,
       -4,  5])

In [17]:
Species = pd.DataFrame(columns=['Cantus','Counter'])
for i in range (n):
    Species = Species.append({'Cantus': cantus_lst[i],
         'Counter': counter_lst[i],
         }, ignore_index=True)

In [18]:
Species

,Cantus,Counter
0,58,67
1,66,69
2,64,71
3,68,76
4,66,74
...,...,...
9995,64,72
9996,56,60
9997,57,61
9998,61,65


In [19]:
Species.to_csv('Species.csv')

In [20]:
df = pd.read_csv('Species.csv', sep=",",index_col = 0)
print(df)

      Cantus  Counter
0         58       67
1         66       69
2         64       71
3         68       76
4         66       74
...      ...      ...
9995      64       72
9996      56       60
9997      57       61
9998      61       65
9999      56       64

[10000 rows x 2 columns]


In [6]:
X = tf.Variable(np.zeros((2,n*2)))  # any data tensor

In [7]:
for i in range(0,2*n,2):
    X[0,i].assign(df['Cantus'][i/2])
    X[1,i].assign(0)
    X[0,i+1].assign(df['Counter'][i/2])
    X[1,i+1].assign(1)
X

<tf.Variable 'Variable:0' shape=(2, 400) dtype=float64, numpy=
array([[49., 52., 56., 64., 54., 63., 53., 65., 56., 60., 54., 61., 55.,
        63., 56., 60., 53., 61., 51., 63., 55., 62., 51., 60., 58., 67.,
        56., 65., 59., 67., 66., 69., 65., 77., 67., 75., 71., 74., 68.,
        72., 70., 74., 66., 70., 65., 68., 66., 75., 67., 74., 63., 71.,
        61., 70., 66., 75., 65., 73., 67., 75., 68., 71., 70., 73., 69.,
        76., 68., 72., 66., 70., 72., 75., 70., 74., 69., 78., 61., 70.,
        63., 72., 61., 73., 64., 72., 61., 73., 62., 70., 63., 71., 64.,
        68., 57., 69., 58., 65., 57., 61., 59., 62., 54., 63., 56., 65.,
        53., 62., 54., 61., 51., 59., 52., 56., 50., 57., 54., 62., 57.,
        61., 55., 58., 48., 56., 50., 59., 57., 66., 56., 65., 58., 62.,
        57., 60., 54., 58., 53., 56., 50., 57., 49., 61., 50., 58., 58.,
        62., 56., 64., 49., 57., 50., 59., 53., 57., 56., 65., 55., 64.,
        59., 62., 56., 59., 55., 62., 52., 64., 60., 68., 59.

In [9]:
tf.transpose(X)[:300,:]

<tf.Tensor: shape=(300, 2), dtype=float64, numpy=
array([[49.,  0.],
       [52.,  1.],
       [56.,  0.],
       [64.,  1.],
       [54.,  0.],
       [63.,  1.],
       [53.,  0.],
       [65.,  1.],
       [56.,  0.],
       [60.,  1.],
       [54.,  0.],
       [61.,  1.],
       [55.,  0.],
       [63.,  1.],
       [56.,  0.],
       [60.,  1.],
       [53.,  0.],
       [61.,  1.],
       [51.,  0.],
       [63.,  1.],
       [55.,  0.],
       [62.,  1.],
       [51.,  0.],
       [60.,  1.],
       [58.,  0.],
       [67.,  1.],
       [56.,  0.],
       [65.,  1.],
       [59.,  0.],
       [67.,  1.],
       [66.,  0.],
       [69.,  1.],
       [65.,  0.],
       [77.,  1.],
       [67.,  0.],
       [75.,  1.],
       [71.,  0.],
       [74.,  1.],
       [68.,  0.],
       [72.,  1.],
       [70.,  0.],
       [74.,  1.],
       [66.,  0.],
       [70.,  1.],
       [65.,  0.],
       [68.,  1.],
       [66.,  0.],
       [75.,  1.],
       [67.,  0.],
       [74.,  1.],


In [12]:
dataset = tf.data.Dataset.from_tensor_slices(tf.transpose(X))
dataset

<TensorSliceDataset shapes: (2,), types: tf.float64>

In [13]:
for item in dataset.take(5):
    print(item)

tf.Tensor([49.  0.], shape=(2,), dtype=float64)
tf.Tensor([52.  1.], shape=(2,), dtype=float64)
tf.Tensor([56.  0.], shape=(2,), dtype=float64)
tf.Tensor([64.  1.], shape=(2,), dtype=float64)
tf.Tensor([54.  0.], shape=(2,), dtype=float64)


In [15]:
np.int(X[0][0])

49

In [ ]:
rnn_units = 12

input_ = keras.layers.Input(shape=(None,))
hidden1 = LSTM(rnn_units, return_sequences=True)(input_)